# YoloV3 implementation and testing using Pytorch

### step 1: Making the network's layers

In [2]:
from __future__ import division
import torch
import torchvision 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import variable
import numpy as np


In [ ]:
def pars_cfg(file_path):
    #read the layers and store every block as a dictionary
    
    file = open(file=file_path, mode='r')
    lines = file.read().split('\n')
    lines = [x for x in lines if (len(x)>0 and x[0] != '#')]
    lines = [x.rstrip().lstrip() for x in lines]
    
    block = {}
    blocks = []
    
    for line in lines:
        if line[0] = "[":
            if len(block) != 0:
                blocks.append(block)
                block = {}
            block["type"] = line[1:]
        else:
            key, value = line.split('=')
            block